# Data analysis

## TOC
- Student info
- Important note
- Special credits
- Required imports and basic setup
- Step 1: Loading the data
   - 1.a: Saving the labels
   - 1.b: Saving dictionary of file paths
- Step 2: Data analysis
   - 2.a: Distribtion of the training data
   - 2.b: A deeper look at the training data
   - 2.c: Feature extraction
   - 2.d: The numerical representation
   

## Student info
- **Name**: Bontinck Lennert
- **StudentID**: 568702
- **Affiliation**: VUB - Master Computer Science: AI

## Important note
In some codeblocks, the code might refer to variables from previous sections, in order to get accurate results code must be run top to bottom without skipping.

## Special credits
Some of the code used in this notebook is adopted or copied from the notebooks supplied in the Kaggle compition. A special thanks is given to Andries Rosseau for supplying us with this helpfull code.

## Required imports and basic setup
All required imports for this file are taken care of once using the following code block. Installing the required libraries is discussed in the README of this GitHub repository. Some basic setup for the used libraries is also taken care of here

In [ ]:
# standard packages used to handle files
import sys
import os 
import glob
import time

# commonly used library for data manipulation
import pandas as pd

# numerical
import numpy as np

# handle images - opencv
import cv2

# machine learning library
import sklearn
import sklearn.preprocessing

# used to serialize python objects to disk and load them back to memory
import pickle

# plotting
import matplotlib.pyplot as plt
from matplotlib import cm as cm

# helper functions
import helpers

# specific helper functions for feature extraction
import features

# tell matplotlib that we plot in a notebook and make images high(er) resolution
%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'svg'}

# used for counting files
import fnmatch

In [ ]:
# datasets  location
dataset_path = '../images/'
# output location:
output_path = './'

In [ ]:
# other path settings
dataset_path_train = os.path.join(dataset_path, 'train')
dataset_path_test = os.path.join(dataset_path, 'test')

features_path = os.path.join(output_path, 'features')
features_path_train = os.path.join(features_path, 'train')
features_path_test = os.path.join(features_path, 'test')

prediction_path = os.path.join(output_path, 'predictions')

# filepatterns to write out features
filepattern_descriptor_train = os.path.join(features_path_train, 'train_features_{}.pkl')
filepattern_descriptor_test = os.path.join(features_path_test, 'test_features_{}.pkl')

# create paths in case they don't exist:
helpers.createPath(features_path)
helpers.createPath(features_path_train)
helpers.createPath(features_path_test)
helpers.createPath(prediction_path)

## Step 1: Loading the data
Before we can start exploring our data and making models we have to setup everything that is needed to access the data.

The data is provided in the Kaggle compition and saved under a folder "test" and "train" inside the folder "images" which is saved in this files parent directory as set up in the previous basic setup step.

### 1.a: Saving the labels
The training data provided is saved inside a folder which name corresponds to the label of the data inside that folder. Thus getting the labels of the data is nothing more then getting all folder names inside the "train" folder.

In [ ]:
train_images_folder_paths = glob.glob(os.path.join(dataset_path_train,'*'))
label_strings = np.sort(np.array([os.path.basename(path) for path in train_images_folder_paths]))
amount_of_labels = label_strings.shape[0]

print("Amount of classes (labels): ", amount_of_labels)
print("\nLabels: ", label_strings)

### 1.b: Saving dictionary of file paths
Since loading all images of the train set into memory at once is hardly feasible, we simply collect all their filepaths and load them on demand. Therefore, we build a dictionary of the filepaths to all our train images, sorted by label. We do the same for our test images.

In [ ]:
train_paths = dict((label_string, helpers.getImgPaths(os.path.join(dataset_path_train, label_string))) 
                   for label_string in label_strings)

test_paths = helpers.getImgPaths(dataset_path_test)

## Step 2: Data analysis
Before rigorously testing different models available it's important to take a look at the data that's supplied to us. This will not only validate the data is correctly loaded in the previous step but will also give us some hint as to what our data looks like, what it's distribution is...

### 2.a: Distribtion of the training data
This part is further discussed in the report of this project, in section: Data distribution.

In [ ]:
# retreiving data for plotting
x = np.arange(amount_of_labels)
x_heights = [len(fnmatch.filter(os.listdir(path), '[^.]*.*')) for path in train_images_folder_paths] 

# providing data to the plotting library
fig, ax = plt.subplots()
plt.bar(x, height=x_heights)

# setting up details of the graph
plt.xticks(x, label_strings)
fig.autofmt_xdate()
plt.ylabel('amount of samples provided')
plt.title('Labeled data distribution')

plt.savefig("../graphs/1-data_analysis-labeled_data_distribution.png", facecolor="white", edgecolor='none')
plt.show()

In [ ]:
total_amount_of_training_imgs = np.sum(x_heights)
print('Total amount of training images: {}'.format(total_amount_of_training_imgs))

### 2.b: A deeper look at the training data
This part is further discussed in the report of this project, in section: Deeper look at the training data

In [ ]:
plt.clf()

fig, axes = plt.subplots(nrows=amount_of_labels, ncols=5, constrained_layout=True, figsize=(10, 10))

[ax.get_xaxis().set_visible(False) for ax_row in axes for ax in ax_row]
[ax.get_yaxis().set_visible(False) for ax_row in axes for ax in ax_row]

for idx, label_string in enumerate(label_strings):
    images = [cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB) for path in train_paths[label_string][:5]]

    for colidx in range(5):
        axes[idx, colidx].imshow(images[colidx])
        if colidx == 2: # if this is the center column
            axes[idx, colidx].set_title(label_string)
            


plt.savefig("../graphs/1-data_analysis-labeled_data_overview.png")
plt.show()

### 2.c: Feature extraction
This part is further discussed in the report of this project, in section: Feature extraction

In [ ]:
plt.clf()

current_image = cv2.imread(train_paths["chicken"][0])
corner_image, corner_coords = features.extractShiTomasiCorners(current_image, num_features=500, min_distance=20, visualize=True)    


plt.imshow(cv2.cvtColor(corner_image, cv2.COLOR_BGR2RGB))
plt.savefig("../graphs/1-data_analysis-POI.png") 
plt.show()

### 2.d: The numerical representation
This part is further discussed in the report of this project, in section: Feature extraction

In [ ]:
descriptor_desired = 'sift'

with open(filepattern_descriptor_train.format(descriptor_desired), 'rb') as pkl_file_train:
    train_features_from_pkl = pickle.load(pkl_file_train)
    
print('Number of encoded train images: {}'.format(len(train_features_from_pkl)))

with open(filepattern_descriptor_test.format(descriptor_desired), 'rb') as pkl_file_test:
    test_features_from_pkl = pickle.load(pkl_file_test)
        
print('Number of encoded test images: {}'.format(len(test_features_from_pkl)))


In [ ]:
amount_of_points_per_image = 30

clustered_codebook = helpers.createCodebook(train_features_from_pkl, codebook_size=amount_of_points_per_image)

In [ ]:
# encode all train images 
train_data = []
train_labels = []

for image_features in train_features_from_pkl:
    bow_feature_vector = helpers.encodeImage(image_features.data, clustered_codebook)
    train_data.append(bow_feature_vector)
    train_labels.append(image_features.label)

In [ ]:
print("Amoount of points available: ", len(train_features_from_pkl[0][2]))
print("Amoount of points requested: ", len(train_data[0]))
print("\nTrain labels: ", train_labels[:5])
print("Train data: \n", train_data[:5])

In [ ]:
#convert train labels to numerical representation
label_encoder = sklearn.preprocessing.LabelEncoder()
label_encoder.fit(label_strings)
train_labels_numerical = label_encoder.transform(train_labels)

# loading in data as PD
data = np.c_[train_data, train_labels_numerical]
feature_names = ["Feature " + str(x + 1) for x in range(amount_of_points_per_image)]
columns = np.append(feature_names, ["target"])
pd_data = pd.DataFrame(data, columns=columns)

pd_data.head()

In [ ]:
pd_data.hist(figsize = (16,16)) # figsize: (width,height)
plt.savefig("../graphs/1-data_analysis-feature_representation.png") 
plt.show()

In [ ]:
def plot_correlation_matrix(df):
    """Takes a pandas dataframe as input"""
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(14,10))
    plt.jet() # set the colormap to jet
    cax = ax.matshow(df.corr(), vmin=-1, vmax=1)

    ticks = list(range(len(df.columns)))
    ax.set_xticks(ticks)
    ax.set_yticks(ticks)

    ax.set_xticklabels(df.columns, rotation=90, horizontalalignment='left')
    ax.set_yticklabels(df.columns)
    
    fig.colorbar(cax, ticks=[-1.0,-0.75,-0.5,-0.25,0.0,0.25,0.5,0.75,1.0])

    plt.tight_layout()
    plt.savefig("../graphs/1-data_analysis-correlation_matrix.png") 
    plt.show()
    
plot_correlation_matrix(pd_data)